In [1]:
import spacy
import random
from spacy.util import minibatch, compounding

In [ ]:
def train_spacy_model(data_dict, output_dir):
    nlp = spacy.blank("pt")  # Cria um modelo em branco para o idioma português
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner)
    
    for entity_label, examples in data_dict.items():
        ner.add_label(entity_label)  # Adiciona os rótulos das entidades com base nas keys do dicionário
        
        for text, annotations in examples:
            entities = annotations["entities"]
            doc = nlp.make_doc(text)
            example = spacy.training.Example.from_dict(doc, annotations)
            ner.add_examples([example])
    
    # Treinamento do modelo
    optimizer = nlp.begin_training()
    for _ in range(20):  # Número de iterações de treinamento
        random.shuffle(ner.examples)
        losses = {}
        batches = minibatch(ner.examples, size=compounding(4.0, 32.0, 1.001))
        for batch in batches:
            nlp.update(batch, sgd=optimizer, drop=0.5, losses=losses)
    
    # Salvando o modelo treinado
    nlp.to_disk(output_dir)
    print("Modelo salvo em", output_dir)

#### Exemplo de dados dos parâmetros:

In [ ]:
data_dict = {
    "motor": [
        ("o motor não está funcionando corretamente", {"entities": [(3, 8, "motor")]})
    ]
}

output_dir = "caminho/para/salvar/modelo"

train_spacy_model(data_dict, output_dir)